# Pipeline Construction

In [1]:
from dao import DataAccess

In [2]:
# Loading Data
import operator

X = DataAccess.get_as_dataframe()
y = X["labels"].apply(operator.itemgetter("alcohol"))

### Importing Models

We're going to try three standards to build models. `SVM`, `LR`, `RF` to create baselines to beat.

In [3]:
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

#### Importing Pipeline Class 

In [4]:
from pipelines.alcohol import AlcoholPipeline
help(AlcoholPipeline)

Help on class AlcoholPipeline in module pipelines.alcohol:

class AlcoholPipeline(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, time_features=None, global_features=None, lsi=False, lsi_n=1000)
 |      :param time_features: default(["dayofweek", "hour", "hourofweek"])
 |      :param lsi: if true, includes the TruncatedSVD() piece
 |  
 |  feature_agepipe(self)
 |      :return: UserAgeMonths
 |  
 |  feature_textpipe(self)
 |      :return: Pipeline(ItemGetter -> TfidfVectorizer -> TruncatedSVD)
 |  
 |  feature_timepipe(self)
 |      :return: Pipeline(ItemGetter -> Timestamp2DatetimeIndex -> DatetimeIndexAttr)
 |  
 |  feature_userpipe(self)
 |      :return: Pipeline(UserGeoVectorizer)
 |  
 |  features(self)
 |      :return: FeatureUnion([
 |          ("text", Pipeline(textpipe)),
 |          ("user", Pipeline(userpipe)),
 |          ("time", Pipeline(timepipe)),
 |          ("age", Pipeline(agepipe)),
 |      ])
 |  
 |  pipeline(self, clf)
 |      :param clf: skle

#### Using Latent Semantic Indexing

In [5]:
%time
clf = AlcoholPipeline(lsi=True).pipeline(LogisticRegression())
clf.fit_transform(X, y)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


<4165x432 sparse matrix of type '<class 'numpy.float64'>'
	with 1662730 stored elements in Compressed Sparse Column format>

#### Using only TFIDF Normalization

In [6]:
%%time
clf = AlcoholPipeline().pipeline(LogisticRegression())
clf.fit_transform(X, y)

CPU times: user 2.59 s, sys: 42.3 ms, total: 2.64 s
Wall time: 2.63 s


## Generating Train-Test Split and Learning Curves

The code for this comes from [Plotting Learning Curves](http://scikit-learn.org/stable/auto_examples/plot_learning_curve.html) by sklearn.

In [7]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from sklearn import cross_validation
from sklearn.learning_curve import learning_curve


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and traning learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : integer, cross-validation generator, optional
        If an integer is passed, it is the number of folds (defaults to 3).
        Specific cross-validation objects can be passed, see
        sklearn.cross_validation module for the list of possible objects

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring="accuracy")
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [8]:
cv = cross_validation.ShuffleSplit(
    len(y), n_iter=20, test_size=0.2, random_state=0
)

# Learning Curves

In [ ]:
import seaborn as sns

sns.set_style("darkgrid")

#### Logistic Regression

With sufficiently low variance we'll see that without tuning

In [ ]:
%%time
title = "Learning Curves (LOGISTIC REGRESSION)"
estimator = AlcoholPipeline(lsi=True, lsi_n=1000).pipeline(LogisticRegression())
plot_learning_curve(estimator, title, AlcoholPipeline()._exploder.fit_transform(X), y, (0.7, 1.01), cv=cv, n_jobs=4)
plt.show()

#### LinearSVC

Variance is really high...

In [ ]:
%%time
title = "Learning Curves (LinearSVC)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
estimator = AlcoholPipeline(lsi=True, lsi_n=1000).pipeline(LinearSVC(
        C=1
    ))
plot_learning_curve(estimator, title, AlcoholPipeline()._exploder.fit_transform(X), y, (0.7, 1.01), cv=cv, n_jobs=4)
plt.show()

#### RandomForests

Notice how well it gets the training data, I feel like this has the most room to improve.

In [ ]:
%%time
title = "Learning Curves (LinearSVC)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
estimator = AlcoholPipeline(lsi=True, lsi_n=1000).pipeline(
    RandomForestClassifier()
    )
plot_learning_curve(estimator, title, AlcoholPipeline()._exploder.fit_transform(X), y, (0.7, 1.01), cv=cv, n_jobs=4)
plt.show()

This is just another one to show that even with a bit of tuning, and falling training score, CV score is still pretty good.

In [ ]:
%%time
title = "Learning Curves (RandomForests)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
estimator = AlcoholPipeline(lsi=True, lsi_n=1000).pipeline(
    RandomForestClassifier(
        n_estimators=200,
        max_depth=50)
    )
plot_learning_curve(estimator, title, AlcoholPipeline()._exploder.fit_transform(X), y, (0.7, 1.01), cv=cv, n_jobs=4)
plt.show()